In [10]:
#checking 100 iter from cmd line train
import os
import torch
os.chdir('/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/src/vendor_run/mellotron')

from hparams import create_hparams
from model import Tacotron2
from text import cmudict, text_to_sequence
from train import warm_start_model
from loss_function import Tacotron2Loss
import waveglow.glow
from waveglow.denoiser import Denoiser
import waveglow
import sys
import IPython.display as ipd
from train import prepare_dataloaders
import time 
sys.path.append('waveglow/')
def to_arpabet(text):
    from g2p_en import G2p
    g2p = G2p()
    return " ".join(
        [
            f"{{ {s.strip()} }}" if s.strip() not in ",." else s.strip()
            for s in " ".join(g2p(text)).split("  ")
        ]
    )

def load_model(hparams):
    model = Tacotron2(hparams).cuda()
    if hparams.fp16_run:
        model.decoder.attention_layer.score_mask_value = finfo('float16').min

    return model


waveglow_path = '../../vendor_tools/waveglow/waveglow_256channels_universal_v4.pt'
waveglow_2 = torch.load(waveglow_path)['model'].cuda().eval()
denoiser = Denoiser(waveglow_2).cuda().eval()

hparams = create_hparams(None)
hparams.n_speakers = 1
hparams.training_files = '/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/experiments/filelists/lj_train.txt'
hparams.validation_files = '/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/experiments/filelists/lj_val.txt'
hparams.batch_size = 16
#hparams.checkpoint_path = '/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/experiments/outputs/lj_929_2/checkpoint_0'

hparams.checkpoint_path = '/mnt/disks/uberduck-experiments-v0/models/mellotron/vendor/mellotron_ljs.pt'
#hparams.checkpoint_path = '/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/experiments/outputs/lj_929/checkpoint_0'
#hparams.checkpoint_path = '/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/experiments/outputs/lj_929/checkpoint_200'
hparams.warm_start = True
learning_rate = 1e-3#
hparams.use_saved_learning_rate = True
torch.manual_seed(hparams.seed)

model = load_model(hparams)
criterion = Tacotron2Loss()
model = warm_start_model(hparams.checkpoint_path, model, hparams.ignore_layers)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate,weight_decay=hparams.weight_decay)
optimizer.eps = 1e-16
osd = optimizer.state_dict()
arpabet_dict = cmudict.CMUDict('data/cmu_dictionary')
train_loader, valset, collate_fn, train_sampler = prepare_dataloaders(hparams)
timestart = time.time()
for epoch in range(1):
    for i, batch in enumerate(train_loader):
        
        x, y = model.parse_batch(batch)
        model.zero_grad()
        y_pred = model(x)
        loss = criterion(y_pred, y)
        print(i, loss)
        reduced_loss = loss.item()
        loss.backward()
        optimizer.step()
        timeend = time.time()
        print(epoch,i, timestart - timeend)
        timestart = timeend
        
        if i > 5:
            break

Warm starting model from checkpoint '/mnt/disks/uberduck-experiments-v0/models/mellotron/vendor/mellotron_ljs.pt'
here
0 tensor(0.3902, device='cuda:0', grad_fn=<AddBackward0>)
0 0 -6.489955425262451
here
1 tensor(4.5549, device='cuda:0', grad_fn=<AddBackward0>)


Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/multiprocessing/queues.py", line 242, in _feed
    send_bytes(obj)
  File "/opt/conda/lib/python3.7/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/opt/conda/lib/python3.7/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/opt/conda/lib/python3.7/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
BrokenPipeError: [Errno 32] Broken pipe


KeyboardInterrupt: 

In [11]:
optimizer.load_state_dict(osd)

In [24]:
hparams = create_hparams(None)
hparams.n_speakers = 1
hparams.training_files = '/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/experiments/filelists/lj_train.txt'
hparams.validation_files = '/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/experiments/filelists/lj_val.txt'
hparams.batch_size = 16
#hparams.checkpoint_path = '/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/experiments/outputs/lj_929_2/checkpoint_0'

hparams.checkpoint_path = '/mnt/disks/uberduck-experiments-v0/models/mellotron/vendor/mellotron_ljs.pt'
#hparams.checkpoint_path = '/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/experiments/outputs/lj_929/checkpoint_0'
#hparams.checkpoint_path = '/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/experiments/outputs/lj_929/checkpoint_200'
hparams.warm_start = True
learning_rate = 1e-3#
hparams.use_saved_learning_rate = True
torch.manual_seed(hparams.seed)

model = load_model(hparams)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate,weight_decay=hparams.weight_decay)
optimizer.eps = 1e-16

model = warm_start_model(hparams.checkpoint_path, model, hparams.ignore_layers)
arpabet_dict = cmudict.CMUDict('data/cmu_dictionary')
train_loader, valset, collate_fn, train_sampler = prepare_dataloaders(hparams)
timestart = time.time()
for epoch in range(1):
    for i, batch in enumerate(train_loader):
        
        x, y = model.parse_batch(batch)
        model.zero_grad()
        y_pred = model(x)
        loss = criterion(y_pred, y)
        print(i, loss)
        reduced_loss = loss.item()
        loss.backward()
        optimizer.step()
        timeend = time.time()
        print(epoch,i, timestart - timeend)
        timestart = timeend
        
        if i > 5:
            break

Warm starting model from checkpoint '/mnt/disks/uberduck-experiments-v0/models/mellotron/vendor/mellotron_ljs.pt'
here
0 tensor(0.3902, device='cuda:0', grad_fn=<AddBackward0>)
0 0 -6.556809186935425
here
1 tensor(4.5549, device='cuda:0', grad_fn=<AddBackward0>)


KeyboardInterrupt: 

In [31]:
hparams = create_hparams(None)
hparams.n_speakers = 1
hparams.training_files = '/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/experiments/filelists/lj_train.txt'
hparams.validation_files = '/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/experiments/filelists/lj_val.txt'
hparams.batch_size = 16
#hparams.checkpoint_path = '/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/experiments/outputs/lj_929_2/checkpoint_0'

hparams.checkpoint_path = '/mnt/disks/uberduck-experiments-v0/models/mellotron/vendor/mellotron_ljs.pt'
#hparams.checkpoint_path = '/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/experiments/outputs/lj_929/checkpoint_0'
#hparams.checkpoint_path = '/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/experiments/outputs/lj_929/checkpoint_200'
hparams.warm_start = True
learning_rate = 1e-5#
hparams.use_saved_learning_rate = True
torch.manual_seed(hparams.seed)

model = load_model(hparams)
model = warm_start_model(hparams.checkpoint_path, model, hparams.ignore_layers)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate,weight_decay=hparams.weight_decay)
arpabet_dict = cmudict.CMUDict('data/cmu_dictionary')
train_loader, valset, collate_fn, train_sampler = prepare_dataloaders(hparams)
timestart = time.time()
for epoch in range(1):
    for i, batch in enumerate(train_loader):
        
        x, y = model.parse_batch(batch)
        model.zero_grad()
        y_pred = model(x)
        loss = criterion(y_pred, y)
        print(i, loss)
        reduced_loss = loss.item()
        loss.backward()
        optimizer.step()
        timeend = time.time()
        print(epoch,i, timestart - timeend)
        timestart = timeend
        
        if i > 5:
            break

Warm starting model from checkpoint '/mnt/disks/uberduck-experiments-v0/models/mellotron/vendor/mellotron_ljs.pt'
here
0 tensor(0.3902, device='cuda:0', grad_fn=<AddBackward0>)
0 0 -6.669978141784668
here
1 tensor(0.3704, device='cuda:0', grad_fn=<AddBackward0>)
0 1 -3.9926352500915527
here
2 tensor(0.4028, device='cuda:0', grad_fn=<AddBackward0>)
0 2 -3.9583635330200195
here
3 tensor(0.3424, device='cuda:0', grad_fn=<AddBackward0>)
0 3 -3.9528937339782715
here
4 tensor(0.3419, device='cuda:0', grad_fn=<AddBackward0>)
0 4 -3.930833578109741
here
5 tensor(0.3567, device='cuda:0', grad_fn=<AddBackward0>)
0 5 -3.865662097930908
here
6 tensor(0.3251, device='cuda:0', grad_fn=<AddBackward0>)
0 6 -3.9066553115844727


In [21]:
hparams.checkpoint_path = '/mnt/disks/uberduck-experiments-v0/models/mellotron/vendor/mellotron_ljs.pt'
hparams.checkpoint_path = '/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/experiments/outputs/lj_929/checkpoint_0'

nm= torch.load(hparams.checkpoint_path)

In [20]:
#nm['state_dict'].keys()

In [23]:
nm['learning_rate']

0.001

In [14]:
hparams = create_hparams(None)
hparams.n_speakers = 1
hparams.training_files = '/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/experiments/filelists/lj_train.txt'
hparams.validation_files = '/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/experiments/filelists/lj_val.txt'
hparams.batch_size = 16
hparams.checkpoint_path = '/mnt/disks/uberduck-experiments-v0/models/mellotron/vendor/mellotron_ljs.pt'
hparams.warm_start = True
hparams.use_saved_learning_rate = True
torch.manual_seed(hparams.seed)
learning_rate = 1e-3

model = load_model(hparams)
model = warm_start_model(hparams.checkpoint_path, model, hparams.ignore_layers)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate,weight_decay=hparams.weight_decay)
#osd = optimizer.state_dict()
optimizer.load_state_dict(osd)
train_loader, valset, collate_fn, train_sampler = prepare_dataloaders(hparams)
timestart = time.time()
for epoch in range(1):
    for i, batch in enumerate(train_loader):
        
        x, y = model.parse_batch(batch)
        model.zero_grad()
        y_pred = model(x)
        loss = criterion(y_pred, y)
        print(i, loss)
        reduced_loss = loss.item()
        loss.backward()
        optimizer.step()
        timeend = time.time()
        print(epoch,i, timestart - timeend)
        timestart = timeend
        
        if i > 5:
            break

Warm starting model from checkpoint '/mnt/disks/uberduck-experiments-v0/models/mellotron/vendor/mellotron_ljs.pt'
here
0 tensor(0.3902, device='cuda:0', grad_fn=<AddBackward0>)
0 0 -6.833431005477905
here
1 tensor(4.5549, device='cuda:0', grad_fn=<AddBackward0>)
0 1 -4.133509635925293
here


KeyboardInterrupt: 

In [ ]:
hparams

In [13]:
torch.__version__

'1.9.0'

In [4]:
hparams = create_hparams(None)
hparams.n_speakers = 1
hparams.training_files = '/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/experiments/filelists/lj_train.txt'
hparams.validation_files = '/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/experiments/filelists/lj_val.txt'
hparams.batch_size = 16
#hparams.checkpoint_path = '/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/experiments/outputs/lj_929_2/checkpoint_0'

hparams.checkpoint_path = '/mnt/disks/uberduck-experiments-v0/models/mellotron/vendor/mellotron_ljs.pt'
#hparams.checkpoint_path = '/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/experiments/outputs/lj_929/checkpoint_0'
#hparams.checkpoint_path = '/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/experiments/outputs/lj_929/checkpoint_200'
hparams.warm_start = True
learning_rate = 1e-3#
hparams.use_saved_learning_rate = True
torch.manual_seed(hparams.seed)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate,weight_decay=hparams.weight_decay)

model = load_model(hparams)
model = warm_start_model(hparams.checkpoint_path, model, hparams.ignore_layers)
arpabet_dict = cmudict.CMUDict('data/cmu_dictionary')
train_loader, valset, collate_fn, train_sampler = prepare_dataloaders(hparams)
timestart = time.time()
for epoch in range(1):
    for i, batch in enumerate(train_loader):
        
        x, y = model.parse_batch(batch)
        model.zero_grad()
        y_pred = model(x)
        loss = criterion(y_pred, y)
        print(i, loss)
        reduced_loss = loss.item()
        loss.backward()
        optimizer.step()
        timeend = time.time()
        print(epoch,i, timestart - timeend)
        timestart = timeend
        
        if i > 5:
            break

Warm starting model from checkpoint '/mnt/disks/uberduck-experiments-v0/models/mellotron/vendor/mellotron_ljs.pt'
here
0 tensor(0.3902, device='cuda:0', grad_fn=<AddBackward0>)
0 0 -7.192556381225586
here
1 tensor(0.3843, device='cuda:0', grad_fn=<AddBackward0>)
0 1 -3.978590965270996
here
2 tensor(0.4269, device='cuda:0', grad_fn=<AddBackward0>)
0 2 -3.98422908782959
here
3 tensor(0.3863, device='cuda:0', grad_fn=<AddBackward0>)
0 3 -4.075099945068359
here
4 tensor(0.3840, device='cuda:0', grad_fn=<AddBackward0>)


KeyboardInterrupt: 

In [5]:

hparams = create_hparams(None)

hparams.n_speakers = 1
hparams.training_files = '/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/experiments/filelists/lj_train.txt'
hparams.validation_files = '/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/experiments/filelists/lj_val.txt'
hparams.batch_size = 16
#hparams.checkpoint_path = '/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/experiments/outputs/lj_929_2/checkpoint_0'

hparams.checkpoint_path = '/mnt/disks/uberduck-experiments-v0/models/mellotron/vendor/mellotron_ljs.pt'
#hparams.checkpoint_path = '/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/experiments/outputs/lj_929/checkpoint_0'
#hparams.checkpoint_path = '/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/experiments/outputs/lj_929/checkpoint_200'
hparams.warm_start = True
learning_rate = 1e-3#
hparams.use_saved_learning_rate = True
torch.manual_seed(hparams.seed)

model = load_model(hparams)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate,weight_decay=hparams.weight_decay)
model = warm_start_model(hparams.checkpoint_path, model, hparams.ignore_layers)
arpabet_dict = cmudict.CMUDict('data/cmu_dictionary')
train_loader, valset, collate_fn, train_sampler = prepare_dataloaders(hparams)
timestart = time.time()
for epoch in range(1):
    for i, batch in enumerate(train_loader):
        
        x, y = model.parse_batch(batch)
        model.zero_grad()
        y_pred = model(x)
        loss = criterion(y_pred, y)
        print(i, loss)
        reduced_loss = loss.item()
        loss.backward()
        optimizer.step()
        timeend = time.time()
        print(epoch,i, timestart - timeend)
        timestart = timeend
        
        if i > 5:
            break

Warm starting model from checkpoint '/mnt/disks/uberduck-experiments-v0/models/mellotron/vendor/mellotron_ljs.pt'
here
0 tensor(0.3902, device='cuda:0', grad_fn=<AddBackward0>)
0 0 -6.632129907608032
here
1 tensor(4.5549, device='cuda:0', grad_fn=<AddBackward0>)


KeyboardInterrupt: 

In [6]:

hparams = create_hparams(None)

hparams.n_speakers = 1
hparams.training_files = '/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/experiments/filelists/lj_train.txt'
hparams.validation_files = '/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/experiments/filelists/lj_val.txt'
hparams.batch_size = 16
#hparams.checkpoint_path = '/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/experiments/outputs/lj_929_2/checkpoint_0'

hparams.checkpoint_path = '/mnt/disks/uberduck-experiments-v0/models/mellotron/vendor/mellotron_ljs.pt'
#hparams.checkpoint_path = '/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/experiments/outputs/lj_929/checkpoint_0'
#hparams.checkpoint_path = '/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/experiments/outputs/lj_929/checkpoint_200'
hparams.warm_start = True
learning_rate = 1e-3#
hparams.use_saved_learning_rate = True
torch.manual_seed(hparams.seed)

model = load_model(hparams)
model = warm_start_model(hparams.checkpoint_path, model, hparams.ignore_layers)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate,weight_decay=hparams.weight_decay)
arpabet_dict = cmudict.CMUDict('data/cmu_dictionary')
train_loader, valset, collate_fn, train_sampler = prepare_dataloaders(hparams)
timestart = time.time()
for epoch in range(1):
    for i, batch in enumerate(train_loader):
        
        x, y = model.parse_batch(batch)
        model.zero_grad()
        y_pred = model(x)
        loss = criterion(y_pred, y)
        print(i, loss)
        reduced_loss = loss.item()
        loss.backward()
        optimizer.step()
        timeend = time.time()
        print(epoch,i, timestart - timeend)
        timestart = timeend
        
        if i > 5:
            break

Warm starting model from checkpoint '/mnt/disks/uberduck-experiments-v0/models/mellotron/vendor/mellotron_ljs.pt'
here
0 tensor(0.3902, device='cuda:0', grad_fn=<AddBackward0>)
0 0 -6.643727779388428
here
1 tensor(4.5549, device='cuda:0', grad_fn=<AddBackward0>)
0 1 -4.004593849182129
here


KeyboardInterrupt: 

In [7]:

hparams = create_hparams(None)

hparams.n_speakers = 1
hparams.training_files = '/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/experiments/filelists/lj_train.txt'
hparams.validation_files = '/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/experiments/filelists/lj_val.txt'
hparams.batch_size = 16
#hparams.checkpoint_path = '/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/experiments/outputs/lj_929_2/checkpoint_0'

hparams.checkpoint_path = '/mnt/disks/uberduck-experiments-v0/models/mellotron/vendor/mellotron_ljs.pt'
#hparams.checkpoint_path = '/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/experiments/outputs/lj_929/checkpoint_0'
#hparams.checkpoint_path = '/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/experiments/outputs/lj_929/checkpoint_200'
hparams.warm_start = True
learning_rate = 1e-3#
hparams.use_saved_learning_rate = True
torch.manual_seed(hparams.seed)

model = load_model(hparams)
model = warm_start_model(hparams.checkpoint_path, model, hparams.ignore_layers)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate,weight_decay=hparams.weight_decay)
arpabet_dict = cmudict.CMUDict('data/cmu_dictionary')
train_loader, valset, collate_fn, train_sampler = prepare_dataloaders(hparams)
timestart = time.time()
for epoch in range(1):
    for i, batch in enumerate(train_loader):
        
        x, y = model.parse_batch(batch)
        model.zero_grad()
        y_pred = model(x)
        loss = criterion(y_pred, y)
        print(i, loss)
        reduced_loss = loss.item()
        loss.backward()
        optimizer.step()
        timeend = time.time()
        print(epoch,i, timestart - timeend)
        timestart = timeend
        
        if i > 5:
            break

Warm starting model from checkpoint '/mnt/disks/uberduck-experiments-v0/models/mellotron/vendor/mellotron_ljs.pt'
here
0 tensor(0.3902, device='cuda:0', grad_fn=<AddBackward0>)
0 0 -6.593294143676758
here
1 tensor(4.5549, device='cuda:0', grad_fn=<AddBackward0>)
0 1 -3.9738752841949463
here
2 tensor(1.8494, device='cuda:0', grad_fn=<AddBackward0>)


KeyboardInterrupt: 

In [8]:

hparams = create_hparams(None)

hparams.n_speakers = 1
hparams.training_files = '/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/experiments/filelists/lj_train.txt'
hparams.validation_files = '/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/experiments/filelists/lj_val.txt'
hparams.batch_size = 16
#hparams.checkpoint_path = '/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/experiments/outputs/lj_929_2/checkpoint_0'

hparams.checkpoint_path = '/mnt/disks/uberduck-experiments-v0/models/mellotron/vendor/mellotron_ljs.pt'
#hparams.checkpoint_path = '/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/experiments/outputs/lj_929/checkpoint_0'
#hparams.checkpoint_path = '/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/experiments/outputs/lj_929/checkpoint_200'
hparams.warm_start = True
learning_rate = 1e-3#
hparams.use_saved_learning_rate = True
torch.manual_seed(hparams.seed)

model = load_model(hparams)
model = warm_start_model(hparams.checkpoint_path, model, hparams.ignore_layers)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate,weight_decay=hparams.weight_decay)
torch.manual_seed(hparams.seed)
arpabet_dict = cmudict.CMUDict('data/cmu_dictionary')
train_loader, valset, collate_fn, train_sampler = prepare_dataloaders(hparams)
timestart = time.time()
for epoch in range(1):
    for i, batch in enumerate(train_loader):
        
        x, y = model.parse_batch(batch)
        model.zero_grad()
        y_pred = model(x)
        loss = criterion(y_pred, y)
        print(i, loss)
        reduced_loss = loss.item()
        loss.backward()
        optimizer.step()
        timeend = time.time()
        print(epoch,i, timestart - timeend)
        timestart = timeend
        
        if i > 5:
            break

Warm starting model from checkpoint '/mnt/disks/uberduck-experiments-v0/models/mellotron/vendor/mellotron_ljs.pt'
here
0 tensor(0.3925, device='cuda:0', grad_fn=<AddBackward0>)
0 0 -6.761454820632935
here
1 tensor(5.9231, device='cuda:0', grad_fn=<AddBackward0>)
0 1 -3.9528398513793945
here
2 tensor(0.9206, device='cuda:0', grad_fn=<AddBackward0>)


KeyboardInterrupt: 

In [14]:

hparams = create_hparams(None)

hparams.n_speakers = 1
hparams.training_files = '/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/experiments/filelists/lj_train.txt'
hparams.validation_files = '/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/experiments/filelists/lj_val.txt'
hparams.batch_size = 16
#hparams.checkpoint_path = '/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/experiments/outputs/lj_929_2/checkpoint_0'

hparams.checkpoint_path = '/mnt/disks/uberduck-experiments-v0/models/mellotron/vendor/mellotron_ljs.pt'
#hparams.checkpoint_path = '/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/experiments/outputs/lj_929/checkpoint_0'
#hparams.checkpoint_path = '/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/experiments/outputs/lj_929/checkpoint_200'
hparams.warm_start = True
learning_rate = 1e-3#
hparams.use_saved_learning_rate = True
torch.manual_seed(hparams.seed)

model = load_model(hparams)
model = warm_start_model(hparams.checkpoint_path, model, hparams.ignore_layers)
torch.manual_seed(hparams.seed)
torch.cuda.manual_seed(hparams.seed)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate,weight_decay=hparams.weight_decay)
torch.manual_seed(hparams.seed)
torch.cuda.manual_seed(hparams.seed)
arpabet_dict = cmudict.CMUDict('data/cmu_dictionary')
train_loader, valset, collate_fn, train_sampler = prepare_dataloaders(hparams)
timestart = time.time()
for epoch in range(1):
    for i, batch in enumerate(train_loader):
        
        x, y = model.parse_batch(batch)
        model.zero_grad()
        y_pred = model(x)
        loss = criterion(y_pred, y)
        print(i, loss)
        reduced_loss = loss.item()
        loss.backward()
        optimizer.step()
        timeend = time.time()
        print(epoch,i, timestart - timeend)
        timestart = timeend
        
        if i > 5:
            break

Warm starting model from checkpoint '/mnt/disks/uberduck-experiments-v0/models/mellotron/vendor/mellotron_ljs.pt'
here
0 tensor(0.3925, device='cuda:0', grad_fn=<AddBackward0>)
0 0 -6.776230573654175
here
1 tensor(5.9231, device='cuda:0', grad_fn=<AddBackward0>)
0 1 -3.9313628673553467
here


KeyboardInterrupt: 

In [15]:

hparams = create_hparams(None)

hparams.n_speakers = 1
hparams.training_files = '/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/experiments/filelists/lj_train.txt'
hparams.validation_files = '/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/experiments/filelists/lj_val.txt'
hparams.batch_size = 16
#hparams.checkpoint_path = '/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/experiments/outputs/lj_929_2/checkpoint_0'

hparams.checkpoint_path = '/mnt/disks/uberduck-experiments-v0/models/mellotron/vendor/mellotron_ljs.pt'
#hparams.checkpoint_path = '/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/experiments/outputs/lj_929/checkpoint_0'
#hparams.checkpoint_path = '/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/experiments/outputs/lj_929/checkpoint_200'
hparams.warm_start = True
learning_rate = 1e-3#
hparams.use_saved_learning_rate = True
torch.manual_seed(hparams.seed)

model = load_model(hparams)
model = warm_start_model(hparams.checkpoint_path, model, hparams.ignore_layers)
torch.manual_seed(hparams.seed)
torch.cuda.manual_seed(hparams.seed)
#optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate,weight_decay=hparams.weight_decay)
torch.manual_seed(hparams.seed)
torch.cuda.manual_seed(hparams.seed)
arpabet_dict = cmudict.CMUDict('data/cmu_dictionary')
train_loader, valset, collate_fn, train_sampler = prepare_dataloaders(hparams)
timestart = time.time()
for epoch in range(1):
    for i, batch in enumerate(train_loader):
        
        x, y = model.parse_batch(batch)
        model.zero_grad()
        y_pred = model(x)
        loss = criterion(y_pred, y)
        print(i, loss)
        reduced_loss = loss.item()
        loss.backward()
        optimizer.step()
        timeend = time.time()
        print(epoch,i, timestart - timeend)
        timestart = timeend
        
        if i > 5:
            break

Warm starting model from checkpoint '/mnt/disks/uberduck-experiments-v0/models/mellotron/vendor/mellotron_ljs.pt'
here
0 tensor(0.3925, device='cuda:0', grad_fn=<AddBackward0>)
0 0 -6.712111711502075
here
1 tensor(0.4015, device='cuda:0', grad_fn=<AddBackward0>)
0 1 -4.010849237442017
here
2 tensor(0.3832, device='cuda:0', grad_fn=<AddBackward0>)


KeyboardInterrupt: 

In [12]:
optimizer.state

defaultdict(dict,
            {Parameter containing:
             tensor([[ 0.0097, -0.0109,  0.0030,  ..., -0.0012, -0.0002, -0.0085],
                     [-0.0715, -0.0675,  0.0661,  ..., -0.0572,  0.0415, -0.0155],
                     [ 0.0530, -0.0406, -0.0093,  ..., -0.0019,  0.0401,  0.0226],
                     ...,
                     [ 0.0225,  0.0069,  0.0200,  ..., -0.0461,  0.0129,  0.0013],
                     [ 0.0173, -0.0168, -0.0004,  ...,  0.0061,  0.0479, -0.0130],
                     [ 0.0098,  0.0031,  0.0168,  ...,  0.0040,  0.0950, -0.0113]],
                    device='cuda:0', requires_grad=True): {'step': 7,
              'exp_avg': tensor([[ 6.0874e-09, -6.7347e-09,  2.1162e-09,  ..., -9.2871e-10,
                       -1.4021e-10, -5.3998e-09],
                      [-3.8930e-08, -3.6852e-08,  3.6127e-08,  ..., -3.1437e-08,
                        2.3174e-08, -9.3027e-09],
                      [ 2.9221e-08, -2.2704e-08, -5.8387e-09,  ..., -1.4278e-09

In [32]:
import torch
from numba import cuda
torch.cuda.empty_cache()
cuda.select_device(0)
cuda.close()